# Tarea 1
## Luis Morera Claramunt

Se importan las librerías necesarias

In [1]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('data_processing').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
import findspark
findspark.init('/Users/Luis/Desktop/spark/spark-2.4.4-bin-hadoop2.7')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

Se carga la base de datos

In [3]:
df=spark.read.csv("2019_opens_athletes.csv",header=True, inferSchema=True)

Numero de filas y columnas del set de datos

In [4]:
print("Filas",df.count())
print("Columnas",len(df.columns))

Filas 572653
Columnas 19


Columnas y tipo de datos que contienen

In [5]:
df.printSchema()

root
 |-- competitorid: integer (nullable = true)
 |-- competitorname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- postcompstatus: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- profilepics3key: string (nullable = true)
 |-- countryoforigincode: string (nullable = true)
 |-- countryoforiginname: string (nullable = true)
 |-- divisionid: integer (nullable = true)
 |-- affiliateid: integer (nullable = true)
 |-- affiliatename: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- overallrank: integer (nullable = true)
 |-- overallscore: integer (nullable = true)
 |-- is_scaled: integer (nullable = true)
 |-- division: string (nullable = true)



Se va a trabaja con 10 columnas para poder tener una mejor visualizacion de la tabla en el notebook

In [6]:
cols_to_use = ['gender', 'countryoforiginname', 'height', 'weight', 'age', 'overallrank', 'division', 'is_scaled']
df = df.select(cols_to_use)

In [7]:
df.show(10)

+------+-------------------+------+------+---+-----------+-------------+---------+
|gender|countryoforiginname|height|weight|age|overallrank|     division|is_scaled|
+------+-------------------+------+------+---+-----------+-------------+---------+
|     F|     United Kingdom|   1.7| 61.23| 37|          1|Women (35-39)|        0|
|     F|             Brazil|  1.74|  73.0| 36|          2|Women (35-39)|        0|
|     F|      United States|  1.57| 62.14| 35|          3|Women (35-39)|        0|
|     F|              Italy|   1.7|  71.0| 35|          4|Women (35-39)|        0|
|     F|      United States|  1.55| 61.23| 36|          5|Women (35-39)|        0|
|     F|      United States|  1.65| 66.68| 36|          6|Women (35-39)|        0|
|     F|             Brazil|   1.7|  72.0| 36|          7|Women (35-39)|        0|
|     F|      United States|  1.75| 68.95| 38|          8|Women (35-39)|        0|
|     F|        Switzerland|  1.64|  62.0| 38|          9|Women (35-39)|        0|
|   

Cantidad de atletas por categoría

In [10]:
df.groupBy('division').count().show()

+-------------+------+
|     division| count|
+-------------+------+
|Women (35-39)| 31934|
|Women (45-49)| 13632|
|Women (50-54)|  7972|
|  Men (45-49)| 17894|
|  Men (50-54)|  9476|
|          Men|210150|
|    Men (60+)|  3832|
|        Women|172687|
|  Men (35-39)| 42248|
|Women (55-59)|  4560|
|Women (40-44)| 21452|
|  Men (55-59)|  5247|
|  Men (40-44)| 28384|
|  Women (60+)|  3185|
+-------------+------+



Se filtra por campeones en categoría RX (es decir, la variable "is_scaled" = 0), y se ordena por categorías para separar categoría femenina de la masculina. Se puede observar como la tendencia es que las probabilidades de que el campeón provenga de norteamérica son muchos más altas que de algúna otra parte del mundo.

In [11]:
total_rxchamps = df.where((df['overallrank']==1) & (df['is_scaled']==0)).sort('division', ascending=True)
total_rxchamps.show()

+------+-------------------+------+------+---+-----------+-------------+---------+
|gender|countryoforiginname|height|weight|age|overallrank|     division|is_scaled|
+------+-------------------+------+------+---+-----------+-------------+---------+
|     M|      United States|   1.7| 88.45| 29|          1|          Men|        0|
|     M|       South Africa|  1.85| 88.45| 35|          1|  Men (35-39)|        0|
|     M|             Canada|  1.86| 90.72| 42|          1|  Men (40-44)|        0|
|     M|      United States|  1.83| 96.16| 49|          1|  Men (45-49)|        0|
|     M|      United States|  1.83| 88.45| 50|          1|  Men (50-54)|        0|
|     M|             Canada|  1.78| 84.82| 55|          1|  Men (55-59)|        0|
|     M|      United States|   1.7| 74.39| 60|          1|    Men (60+)|        0|
|     F|            Iceland|  1.73|  69.0| 26|          1|        Women|        0|
|     F|     United Kingdom|   1.7| 61.23| 37|          1|Women (35-39)|        0|
|   

En la categoría modificado se muestran resultados similares, sin embargo, es interesante resaltar como en la categoría femenina hay un claro dominio por parte de Estados Unidos. 

In [12]:
total_scaledchamps = df.where((df['division']=='Men') & (df['is_scaled']==1)).sort('division', ascending=True)
total_scaledchamps.show()

+------+-------------------+------+------+---+-----------+--------+---------+
|gender|countryoforiginname|height|weight|age|overallrank|division|is_scaled|
+------+-------------------+------+------+---+-----------+--------+---------+
|     M|              Italy|  1.77|  81.0| 20|          1|     Men|        1|
|     M|             Canada|  1.75| 81.65| 43|          2|     Men|        1|
|     M|             Canada|  null|  null| 30|          3|     Men|        1|
|     M|      United States|   1.7| 76.66| 35|          4|     Men|        1|
|     M|      United States|  1.75| 73.94| 31|          5|     Men|        1|
|     M|     United Kingdom|  null|  83.0| 27|          6|     Men|        1|
|     M|             Sweden|  1.72|  80.0| 27|          7|     Men|        1|
|     M|      United States|  null|  null| 44|          8|     Men|        1|
|     M| Russian Federation|  1.83|  91.0| 22|          9|     Men|        1|
|     M|            Ukraine|  null|  null| 30|         10|     M

Se descrubrió que dentro de los mejores 50 atletas de cada categoría, no hay diferencia significativa de peso ni altura entre las categorías RX y modificado. 

In [13]:
df.filter(df['overallrank']>50).groupBy('is_scaled', 'division').agg(F.avg('height'), F.avg('weight')).sort('division', ascending=True).show()

+---------+-------------+------------------+-----------------+
|is_scaled|     division|       avg(height)|      avg(weight)|
+---------+-------------+------------------+-----------------+
|        1|          Men|1.7824734513274212|85.42082147838241|
|        0|          Men| 1.794954029605374|85.15681010861435|
|        1|  Men (35-39)|1.7726586345381496|87.19644274809141|
|        0|  Men (35-39)|1.7980255905512568|86.26788972064949|
|        0|  Men (40-44)|1.8032583675866107|86.85001411892374|
|        1|  Men (40-44)|1.7736781609195387|87.83451747088172|
|        0|  Men (45-49)|1.7866139447823863|86.95790570461126|
|        1|  Men (45-49)|1.7592940038684723|87.38488728584302|
|        1|  Men (50-54)| 1.791476251604621|88.68226840855093|
|        0|  Men (50-54)|1.7848454746136746|86.84524156636708|
|        0|  Men (55-59)|1.7764210526315762|85.71646047072977|
|        1|  Men (55-59)|1.7292233009708728|89.95551622418897|
|        1|    Men (60+)|1.7613429256594697|84.79444924

Se ordenan los datos para observar los paises que aportaron mayor cantidad de participantes

In [20]:
df.groupBy('countryoforiginname').count().sort('count', ascending=False).show(30)

+-------------------+------+
|countryoforiginname| count|
+-------------------+------+
|      United States|302505|
|             Canada| 31437|
|             Brazil| 30226|
|     United Kingdom| 29020|
|          Australia| 28781|
|             France| 23364|
|            Germany|  9205|
|              Spain|  8157|
|              Italy|  7700|
|        Netherlands|  7478|
|       South Africa|  7294|
|             Sweden|  6699|
|        New Zealand|  5515|
|             Norway|  5099|
| Korea, Republic of|  5055|
|            Ireland|  4411|
|        Switzerland|  4375|
|            Denmark|  3788|
|             Mexico|  3732|
|            Finland|  3618|
| Russian Federation|  3078|
|            Belgium|  2486|
|              China|  2288|
|           Portugal|  2241|
|              India|  2211|
|            Iceland|  2138|
|            Austria|  1605|
|          Guatemala|  1438|
|              Japan|  1207|
|         Costa Rica|  1173|
+-------------------+------+
only showing t

### Comandos no vistos en clase

El comando when es utilizado para evaluar una o más condiciones y devuelve un resultado asociado a ese criterio. Es útil a la hora de transformar los datos, por ejemplo, pasar de datos categoricos a numericos. A continuación se utiliza el comando para pasar la variable de genero a numerica.

Además del comando when, también se utiliza el comando withColumn, el cual crea una nueva columna a partir de los resultados obtenidos. Es util en el caso de no querer perder la información original y agregar los nuevos datos.

In [34]:
df.withColumn('Cat_gender', F.when(df['gender'] =='M', 1).when(df['gender'] =='F', 1)).show()

+------+-------------------+------+------+---+-----------+-------------+---------+----------+
|gender|countryoforiginname|height|weight|age|overallrank|     division|is_scaled|Cat_gender|
+------+-------------------+------+------+---+-----------+-------------+---------+----------+
|     F|     United Kingdom|   1.7| 61.23| 37|          1|Women (35-39)|        0|         1|
|     F|             Brazil|  1.74|  73.0| 36|          2|Women (35-39)|        0|         1|
|     F|      United States|  1.57| 62.14| 35|          3|Women (35-39)|        0|         1|
|     F|              Italy|   1.7|  71.0| 35|          4|Women (35-39)|        0|         1|
|     F|      United States|  1.55| 61.23| 36|          5|Women (35-39)|        0|         1|
|     F|      United States|  1.65| 66.68| 36|          6|Women (35-39)|        0|         1|
|     F|             Brazil|   1.7|  72.0| 36|          7|Women (35-39)|        0|         1|
|     F|      United States|  1.75| 68.95| 38|          8|Wo

Utlizando el comando distinct() podemos saber la cantidad de valores diferentes de una columna. En este caso es conveniente para saber la cantidad de paises diferentes de los cuales participaron atletas

In [40]:
df.select('countryoforiginname').distinct().count()

158